In [1]:
%load_ext Cython

In [2]:
import numpy

In [3]:
%%cython -a

import cython
import numpy
cimport numpy as cnumpy

from libc.math cimport fabs
cdef double EPSILON = numpy.finfo(numpy.float64).eps

cdef cnumpy.int8_t[2] *EDGE_TO_POINT = [[0, 0], [1, 0], [1, 1], [0, 1], [0, 0]]
cdef cnumpy.int8_t[5] *CELL_TO_EDGE = [
                                       # array of index containing
                                       # id0: number of segments (up to 2)
                                       # id1: index of the start of the 1st edge
                                       # id2: index of the end of the 1st edge
                                       # id3: index of the start of the 2nd edge
                                       # id4: index of the end of the 2nd edge
                                       [0, 0, 0, 0, 0],  # Case 0: 0000: nothing
                                       [1, 0, 3, 0, 0],  # Case 1: 0001
                                       [1, 0, 1, 0, 0],  # Case 2: 0010
                                       [1, 1, 3, 0, 0],  # Case 3: 0011

                                       [1, 1, 2, 0, 0],  # Case 4: 0100
                                       [2, 0, 1, 2, 3],  # Case 5: 0101 > ambiguous
                                       [1, 0, 2, 0, 0],  # Case 6: 0110
                                       [1, 2, 3, 0, 0],  # Case 7: 0111

                                       [1, 2, 3, 0, 0],  # Case 8: 1000
                                       [1, 0, 2, 0, 0],  # Case 9: 1001
                                       [2, 0, 3, 1, 2],  # Case 10: 1010 > ambiguous
                                       [1, 1, 2, 0, 0],  # Case 11: 1011

                                       [1, 1, 3, 0, 0],  # Case 12: 1100
                                       [1, 0, 1, 0, 0],  # Case 13: 1101
                                       [1, 0, 3, 0, 0],  # Case 14: 1110
                                       [0, 0, 0, 0, 0],  # Case 15: 1111
                                      ]


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef marching_squares(cnumpy.float32_t[:, :] image, double isovalue, cnumpy.int8_t[:, :] mask=None):
    cdef:
        bint do_mask = mask is not None
        int dim_y = image.shape[0]
        int dim_x = image.shape[1]
        cnumpy.uint8_t[:, :] indexes = numpy.zeros((dim_y - 1, dim_x - 1), dtype=numpy.uint8)
        int x, y, i_segment, i_side, i_edge, index, indexes_count = 0
        double tmpf
    if do_mask:
        assert image.shape[0] == mask.shape[0], "mask shape dim0"
        assert image.shape[1] == mask.shape[1], "mask shape dim1"
    with nogil:
        for y in range(dim_y - 1):
            for x in range(dim_x - 1):

                # Calculate index.
                index = 0
                if image[y, x] > isovalue:
                    index += 1
                if image[y, x + 1] > isovalue:
                    index += 2
                if image[y + 1, x + 1] > isovalue:
                    index += 4
                if image[y + 1, x] > isovalue:
                    index += 8

                # Resolve ambiguity
                if index == 5 or index == 10:
                    # Calculate value of cell center (i.e. average of corners)
                    tmpf = 0.25 * (image[y, x] +
                                   image[y, x + 1] +
                                   image[y + 1, x] +
                                   image[y + 1, x + 1])
                    # If below isovalue, swap
                    if tmpf <= isovalue:
                        if index == 5:
                            index = 10
                        else:
                            index = 5

                # Cache mask information
                if do_mask:
                    if mask[y, x] > 0:
                        index += 16
                    if mask[y, x + 1] > 0:
                        index += 32
                    if mask[y + 1, x + 1] > 0:
                        index += 64
                    if mask[y + 1, x] > 0:
                        index += 128

                if index < 16 and index != 0 and index != 15:
                    indexes[y, x] = index
    return indexes


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef compute_point(
        cnumpy.float32_t[:, :] image,
        cnumpy.uint_t x,
        cnumpy.uint_t y,
        cnumpy.uint8_t edge,
        double isovalue,
        cnumpy.float32_t *result):
    cdef:
        int dx1, dy1, dx2, dy2
        double fx, fy, ff, weight1, weight2
    # Use these to look up the relative positions of the pixels to interpolate
    dx1, dy1 = EDGE_TO_POINT[edge][0], EDGE_TO_POINT[edge][1]
    dx2, dy2 = EDGE_TO_POINT[edge + 1][0], EDGE_TO_POINT[edge + 1][1]
    # Define "strength" of each corner of the cube that we need
    weight1 = 1.0 / (EPSILON + fabs(image[y + dy1, x + dx1] - isovalue))
    weight2 = 1.0 / (EPSILON + fabs(image[y + dy2, x + dx2] - isovalue))
    # Apply a kind of center-of-mass method
    fx, fy, ff = 0.0, 0.0, 0.0
    fx += <double> dx1 * weight1;
    fy += <double> dy1 * weight1;
    ff += weight1
    fx += <double> dx2 * weight2;
    fy += <double> dy2 * weight2;
    ff += weight2
    fx /= ff
    fy /= ff
    result[0] = x + fx
    result[1] = y + fy


cdef struct next_segment_t:
    int x
    int y
    int index
    int edge


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef next_segment(cnumpy.float32_t[:, :] image, cnumpy.uint8_t[:,:] indexes, cnumpy.int_t x, cnumpy.int_t y, cnumpy.uint8_t index, cnumpy.uint8_t edge, next_segment_t *result):
    cdef:
        int next_x, next_y, next_edge, next_index
        cnumpy.int8_t *edges

    index = index & 0x0F
    if  index == 0 or index == 15:
        result.x = -1
        return

    # clean up the cache
    if index == 5:
        if edge == 0 or edge == 1:
            # it's the first segment
            index = 2
            indexes[y, x] = 7
        else:
            # it's the second segment
            index = 8
            indexes[y, x] = 13
    elif index == 10:
        if edge == 0 or edge == 3:
            # it's the first segment
            index = 14
            indexes[y, x] = 4
        else:
            # it's the second segment
            index = 4
            indexes[y, x] = 1
    else:
        indexes[y, x] = 0

    # next
    if edge == 0:
        next_x = x
        next_y = y - 1
    elif edge == 1:
        next_x = x + 1
        next_y = y
    elif edge == 2:
        next_x = x
        next_y = y + 1
    elif edge == 3:
        next_x = x - 1
        next_y = y
    else:
        assert False, "Unexpected behaviour"
    if next_x >= indexes.shape[1] or next_y >= indexes.shape[0] or next_x < 0 or next_y < 0:
        # out of the indexes
        result.x = -1
        return

    next_index = indexes[next_y, next_x]
    next_index = next_index & 0x0F
    if next_index == 0 or next_index == 15:
        # nothing anymore
        result.x = -1
        return

    # top became down, up be came down
    from_edge = edge + 2 if edge < 2 else edge - 2
    edges = CELL_TO_EDGE[next_index]
    if next_index == 5 or next_index == 10:
        # the targeted side is not from_side but the other (from the same segment)
        if edges[1] == from_edge:
            next_edge = edges[2]
        elif edges[2] == from_edge:
            next_edge = edges[1]
        elif edges[3] == from_edge:
            next_edge = edges[4]
        elif edges[4] == from_edge:
            next_edge = edges[3]
    else:
        # the targeted side is not from_side but the other
        next_edge = edges[1] if edges[1] != from_edge else edges[2]

    x, y, index, edge = next_x, next_y, next_index, next_edge

    result.x = x
    result.y = y
    result.index = index
    result.edge = edge
    return


cdef cnumpy.float32_t[:, :] cached_points = numpy.zeros((1024 * 4, 2), numpy.float32)


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef float[:, :] extract_polygon(cnumpy.float32_t[:, :] image, cnumpy.uint8_t[:, :] indexes, double isovalue, int x, int y):
    cdef:
        int ifrom, ito
        cnumpy.uint8_t index, edge
        cnumpy.float32_t *point = [0, 0]
        next_segment_t first_pos
        next_segment_t result

    ifrom = 2048
    ito = 2048
    index = indexes[y, x]
    index = index & 0x0F

    edge = CELL_TO_EDGE[index][1 + 0]
    first_pos.x = x
    first_pos.y = y
    first_pos.index = index
    first_pos.edge = edge
    compute_point(image, x, y, edge, isovalue, point)
    cached_points[ito][0] = point[0]
    cached_points[ito][1] = point[1]
    ito += 1

    edge = CELL_TO_EDGE[index][1 + 1]
    compute_point(image, x, y, edge, isovalue, point)
    cached_points[ito][0] = point[0]
    cached_points[ito][1] = point[1]
    ito += 1

    while True:
        next_segment(image, indexes, x, y, index, edge, &result)
        if result.x < 0:
            break
        x, y, index, edge = result.x, result.y, result.index, result.edge
        compute_point(image, x, y, edge, isovalue, point)
        cached_points[ito][0] = point[0]
        cached_points[ito][1] = point[1]
        ito += 1

    x, y, index, edge = first_pos.x, first_pos.y, first_pos.index, first_pos.edge
    while True:
        next_segment(image, indexes, x, y, index, edge, &result)
        if result.x < 0:
            break
        x, y, index, edge = result.x, result.y, result.index, result.edge
        compute_point(image, x, y, edge, isovalue, point)
        ifrom -= 1
        cached_points[ifrom][0] = point[0]
        cached_points[ifrom][1] = point[1]

    return numpy.array(cached_points[ifrom:ito], dtype=numpy.float32)


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
cdef extract_polygons(cnumpy.float32_t[:, :] image, cnumpy.uint8_t[:, :] indexes, double isovalue):
    cdef:
        int x, y
        cnumpy.uint8_t index
    polygons = []
    with nogil:
        for y in range(indexes.shape[0]):
            for x in range(indexes.shape[1]):
                index = indexes[y, x]
                index = index & 0x0F
                if index == 0 or index == 15:
                    continue
                with gil:
                    polygon = extract_polygon(image, indexes, isovalue, x, y)
                    polygons.append(polygon)

                if index == 5 or index == 10:
                    index = indexes[y, x]
                    index = index & 0x0F
                    if index == 0 or index == 15:
                        continue
                    # There is maybe a second polygon to extract
                    with gil:
                        polygon = extract_polygon(image, indexes, isovalue, x, y)
                        polygons.append(polygon)

    return polygons


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def iso_contour(image=None, value=None, mask=None):
    image = numpy.ascontiguousarray(image, numpy.float32)
    if mask is not None:
        mask = numpy.ascontiguousarray(mask, numpy.int8)
    value = float(value)
    indexes = marching_squares(image, value, mask=mask)
    polygons = extract_polygons(image, indexes, value)
    return polygons

In [4]:
image = numpy.array([[0, 0, 0, 0, 0, 0],
                     [0, 1, 0, 0, 0, 1],
                     [0, 0, 1, 0, 0, 1],
                     [0, 0, 0, 0, 0, 1]], dtype=numpy.float32)

indexes = marching_squares(image, 0.5)
print(numpy.array(indexes))

NameError: name 'marching_squares' is not defined

In [ ]:
image = numpy.array([[0, 0, 0, 0, 0, 0],
                     [0, 0, 1, 1, 0, 0],
                     [0, 1, 1, 1, 1, 0],
                     [0, 0, 1, 0, 0, 0],
                     [0, 0, 0, 0, 0, 0]], dtype=numpy.float32)

indexes = marching_squares(image, 0.5)
print(numpy.array(indexes))

In [ ]:
polygons = extract_polygons(image, indexes.copy(), 0.5)
print(len(polygons))
print(polygons)

In [ ]:
iso_contour(image, value=0.5)

In [ ]:
class MarchingSquareTest(object):

    def __init__(self, image, mask=None):
        self._image = numpy.ascontiguousarray(image, dtype=numpy.float32)
        self._mask = numpy.ascontiguousarray(mask, dtype=numpy.int8)
        print("image %s -> %s; mask %s -> %s" % (image.dtype, self._image.dtype, mask.dtype, self._mask.dtype))

    def iso_contour(self, value):
        value = value.astype(dtype=numpy.float32)
        polygons = iso_contour(self._image, value, mask=self._mask)
        return polygons

In [ ]:
def plot_problem(problem, marching_square):
    import matplotlib
    from matplotlib.patches import Polygon
    from matplotlib.collections import PatchCollection
    import matplotlib.pyplot as plt
    matplotlib.interactive(True)

    fig, ax = plt.subplots()
    ax.set_xmargin(0.1)
    ax.set_ymargin(0.1)
    ax.set_ylim([0, problem.image.shape[0]])
    ax.set_xlim([0, problem.image.shape[1]])
    ax.invert_yaxis()

    # image
    plt.imshow(problem.image, cmap="Greys", alpha=.5)
    
    # mask
    mask = numpy.ma.masked_where(problem.mask == 0, problem.mask)
    plt.imshow(mask, cmap="cool", alpha=.5)

    # iso contours
    colors = ["#FF0000", "#FF7F00", "#FFFF00", "#00FF00", "#0000FF", "#4B0082", "#9400D3"]
    for ivalue, value in enumerate(problem.values):
        color = colors[ivalue % len(colors)]
        polygons = marching_square.iso_contour(value)
        for p in polygons:
            p = Polygon(p, fill=False, edgecolor=color, closed=False)
            ax.add_patch(p)

    plt.show()

In [ ]:
import numpy
import collections
import fabio
Problem = collections.namedtuple("Problem", ["image", "mask", "values"])

def create_problem2():
    ROOT = "/workspace/valls/pyfai.git/_own/iso"
    data = numpy.load(ROOT + "/wos_tth.npz")
    image = data["tth"]
    mask = fabio.open(ROOT + "/wos_mask.edf").data
    mask = (mask != 0)
    values = data["angles"]
    return Problem(image, mask, values)

problem2 = create_problem2()

In [ ]:
from marching_squares import MarchingSquareMPL
marching_square = MarchingSquareMPL(problem2.image, problem2.mask)
%timeit [marching_square.iso_contour(angle) for angle in problem2.values]

In [ ]:
marching_square = MarchingSquareTest(problem2.image, problem2.mask)
%timeit [marching_square.iso_contour(a) for a in problem2.values]

In [ ]:
from marching_squares import MarchingSquareMPL
marching_square = MarchingSquareMPL(problem2.image, problem2.mask)
plot_problem(problem2, marching_square)
marching_square = MarchingSquareTest(problem2.image, problem2.mask)
plot_problem(problem2, marching_square)